# Day 14: Regolith Reservoir

[*Advent of Code 2022 day 14*](https://adventofcode.com/2022/day/14) and [*solution megathread*](https://redd.it/...)

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/UncleCJ/advent-of-code/blob/cj/2022/14/code.ipynb) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/UncleCJ/advent-of-code/cj?filepath=2022%2F14%2Fcode.ipynb)

In [1]:
from IPython.display import HTML
import sys
sys.path.append('../../')


%load_ext nb_mypy
%nb_mypy On

Version 1.0.4


In [2]:
import common


downloaded = common.refresh()
%store downloaded >downloaded

%load_ext pycodestyle_magic
%pycodestyle_on

Writing 'downloaded' (dict) to file 'downloaded'.


## Part One

In [3]:
from IPython.display import HTML

HTML(downloaded['part1'])

## Comments

...

In [4]:
testdata = """498,4 -> 498,6 -> 496,6
503,4 -> 502,4 -> 502,9 -> 494,9""".splitlines()

inputdata = downloaded['input'].splitlines()
# inputdata = open('input.txt', 'r').read().splitlines()

In [5]:
from IPython.display import display


display(f'{inputdata[:10]} ... {len(inputdata)=}')

"['529,71 -> 529,72 -> 539,72 -> 539,71', '484,168 -> 489,168', '493,23 -> 493,13 -> 493,23 -> 495,23 -> 495,19 -> 495,23 -> 497,23 -> 497,15 -> 497,23 -> 499,23 -> 499,14 -> 499,23 -> 501,23 -> 501,14 -> 501,23 -> 503,23 -> 503,21 -> 503,23 -> 505,23 -> 505,18 -> 505,23', '502,165 -> 507,165', '481,165 -> 486,165', '527,96 -> 527,98 -> 525,98 -> 525,106 -> 536,106 -> 536,98 -> 530,98 -> 530,96', '528,68 -> 528,59 -> 528,68 -> 530,68 -> 530,63 -> 530,68 -> 532,68 -> 532,64 -> 532,68 -> 534,68 -> 534,66 -> 534,68 -> 536,68 -> 536,60 -> 536,68', '493,23 -> 493,13 -> 493,23 -> 495,23 -> 495,19 -> 495,23 -> 497,23 -> 497,15 -> 497,23 -> 499,23 -> 499,14 -> 499,23 -> 501,23 -> 501,14 -> 501,23 -> 503,23 -> 503,21 -> 503,23 -> 505,23 -> 505,18 -> 505,23', '500,176 -> 505,176', '513,124 -> 513,126 -> 510,126 -> 510,134 -> 524,134 -> 524,126 -> 518,126 -> 518,124'] ... len(inputdata)=155"

In [6]:
from typing import Iterable, List, Tuple

Coord = Tuple[int, int]
Line = Tuple[Coord, ...]


def parse_lines(data: Iterable[str]) -> List[Line]:
    output: List[Line] = list()
    for line in data:
        line_coords: List[Coord] = list()
        for coord in line.split(' -> '):
            x, y = coord.split(',', 1)
            line_coords.append((int(x), int(y)))
        output.append(tuple(line_coords))
    return output

In [7]:
from typing import Set, Iterable


def generate_lines(lines: Iterable[Line]) -> Set[Coord]:
    def generate_pixels(dim_start: int, dim_end) -> Iterable[int]:
        step = 1
        if dim_start >= dim_end:
            step = -step
        return range(dim_start, dim_end + step, step)

    def generate_segment(c_start: Coord, c_end: Coord) -> Iterable[Coord]:
        if c_start[0] == c_end[0]:
            return ((c_start[0], c1)
                    for c1 in generate_pixels(c_start[1], c_end[1]))
        elif c_start[1] == c_end[1]:
            return ((c0, c_start[1])
                    for c0 in generate_pixels(c_start[0], c_end[0]))
        else:
            raise ValueError

    def generate_line(coords: Line) -> Iterable[Coord]:
        return (pixel
                for c_start, c_end in zip(coords[:-1], coords[1:])
                for pixel in generate_segment(c_start, c_end))

    return {pixel
            for line in lines
            for pixel in generate_line(line)}

In [8]:
def scene_str(stone: Set[Coord], occupied: Set[Coord] = set()) -> List[str]:
    def window(coords: Set[Coord]) -> Tuple[Coord, Coord]:
        min_x = min(c[0] for c in coords)
        max_x = max(c[0] for c in coords)
        min_y = min(c[1] for c in coords)
        max_y = max(c[1] for c in coords)
        return ((min_x - 1, min_y - 1), (max_x + 1, max_y + 1))

    def pixel_chr(pixel: Coord) -> str:
        if pixel in stone:
            return '#'
        elif pixel in occupied:
            return 'o'
        else:
            return '.'

    # replace with occupied.union(stone)?
    w = window(stone.union(occupied))
    return [
        ''.join(pixel_chr((x, y)) for x in range(w[0][0], w[1][0] + 1))
        for y in range(w[0][1], w[1][1] + 1)]

In [9]:
from typing import Optional


def drop_grain(
        stone: Set[Coord],
        occupied: Set[Coord] = set(),
        pos: Coord = (500, 0),
        abyss: int = -1,
        floor_not_abyss: bool = False) -> Optional[Coord]:
    if abyss == -1:
        abyss = max(c[1] + 2 for c in stone)
    # Let's see, if either of the next positions are free,
    # in that order of preference, the grain will fall
    # there, and only if not, it will come to rest where it
    # is
    if floor_not_abyss and pos[1] + 1 >= abyss:
        return pos
    if pos[1] >= abyss or pos in occupied:
        return None
    next_positions = [
        (pos[0], pos[1] + 1),
        (pos[0] - 1, pos[1] + 1),
        (pos[0] + 1, pos[1] + 1)]
    for next_pos in next_positions:
        if next_pos not in occupied:
            return drop_grain(stone,
                              occupied,
                              next_pos,
                              abyss,
                              floor_not_abyss)
    return pos

In [10]:
stone = generate_lines(parse_lines(testdata))
occupied = stone.copy()
while True:
    if position := drop_grain(stone, occupied):
        occupied.add(position)
    else:
        break
display(len(occupied) - len(stone))
display(scene_str(stone, occupied))

24

['............',
 '.......o....',
 '......ooo...',
 '.....#ooo##.',
 '....o#ooo#..',
 '...###ooo#..',
 '.....oooo#..',
 '..o.ooooo#..',
 '.#########..',
 '............']

In [11]:
stone = generate_lines(parse_lines(inputdata))
occupied = stone.copy()
while True:
    if position := drop_grain(stone, occupied):
        occupied.add(position)
    else:
        break
display(len(occupied) - len(stone))

755

In [12]:
HTML(downloaded['part1_footer'])

## Part Two

In [13]:
HTML(downloaded['part2'])

In [14]:
stone = generate_lines(parse_lines(testdata))
occupied = stone.copy()
while True:
    if position := drop_grain(stone, occupied, floor_not_abyss=True):
        occupied.add(position)
    else:
        break
display(len(occupied) - len(stone))
display(scene_str(stone, occupied))

93

['.......................',
 '...........o...........',
 '..........ooo..........',
 '.........ooooo.........',
 '........ooooooo........',
 '.......oo#ooo##o.......',
 '......ooo#ooo#ooo......',
 '.....oo###ooo#oooo.....',
 '....oooo.oooo#ooooo....',
 '...oooooooooo#oooooo...',
 '..ooo#########ooooooo..',
 '.ooooo.......ooooooooo.',
 '.......................']

In [15]:
stone = generate_lines(parse_lines(inputdata))
occupied = stone.copy()
while True:
    if position := drop_grain(stone, occupied, floor_not_abyss=True):
        occupied.add(position)
    else:
        break
display(len(occupied) - len(stone))

29805

In [16]:
HTML(downloaded['part2_footer'])